https://mikrodat.ujbuda.hu/web/

root/mappa_uuid
* ülés_uuid
    * napirendi_uuid  
        * Előterjesztés
        * Döntési javaslatok <- nem kell
        * Mellékletek <- nem kell  
    * napirendi_uuid  
    * ...  

In [ ]:
import sqlite3
import requests
import warnings
import time
import pymupdf
import os
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
warnings.filterwarnings("ignore")

In [ ]:
@dataclass
class Onkorm:
    name: str
    base_url: str
    db_folder: str
    db_detail: str
    db_biz: str
    db_test: str
    db_napirendi: str
    db_test_dokumentum : str
    db_test_melleklet : str
    db_biz_dokumentum : str
    db_biz_melleklet : str        
    db_jegyzokv: str
    db_jegyz_dok: str
    db_hatarozat: str

ujbuda = Onkorm(name = "Újbuda",
                base_url = "https://mikrodat.ujbuda.hu/app/cms/api/honlap", 
                db_folder="ujbuda_meghivo_mappa",
                db_detail="ujbuda_meghivo_reszlet",
                db_test = "ujbuda_test_meghivo",
                db_biz = "ujbuda_biz_meghivo",
                db_napirendi = "ujbuda_napirendi",
                db_test_dokumentum = "ujbuda_test_dokumentum",
                db_test_melleklet = "ujbuda_test_melleklet",
                db_biz_dokumentum = "ujbuda_biz_dokumentum",
                db_biz_melleklet = "ujbuda_biz_melleklet",
                db_jegyzokv = "ujbuda_jegyzokonyv",
                db_jegyz_dok = "ujbuda_jegy_dok",
                db_hatarozat = "ujbuda_hatarozatok",
               )

DATABASE_PATH = "../onkorm.db"
root_folder = Path("../pdf")
output_folder = Path("../txt")

####  Get years

In [ ]:
year_url = f"{ujbuda.base_url}/inv/years"
inv_year_r = requests.get(year_url, verify=False)
years = inv_year_r.json()['content']

#### Database

In [ ]:
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

create_folder_sql = f"""
CREATE TABLE IF NOT EXISTS {ujbuda.db_folder}(
    datum TEXT,
    nyilvanossagjelolo TEXT,
    kategoria TEXT,
    idopont TEXT,
    hely TEXT,
    folder_uuid TEXT UNIQUE
);
"""

folder_detail_table_sql = f"""
CREATE TABLE IF NOT EXISTS {ujbuda.db_detail}(
    datum TEXT, gyujto TEXT, kategoria TEXT, folapra TEXT, eloterjeszto TEXT,
    targy TEXT, napirend TEXT, uuid TEXT PRIMARY KEY, nyilvanossagjelolo TEXT,
    testuletijelolo TEXT, dateLastModified TEXT, name TEXT, idopont TEXT,
    hely TEXT, nev TEXT, iktatoszam TEXT, folder_uuid TEXT
);
"""

agenda_table_sql = f"""
CREATE TABLE IF NOT EXISTS {ujbuda.db_napirendi}(
    gyujto TEXT, nyilvanossagjelolo TEXT, hasPermissions TEXT, folapra TEXT,
    eloterjeszto TEXT, targy TEXT, name TEXT, napirend TEXT, uuid TEXT PRIMARY KEY,
    linkName TEXT, referencia TEXT, folder_uuid TEXT
);
"""

def create_table(sql):
    cursor.execute(sql)
    conn.commit()

# Create necessary tables
create_table(create_folder_sql)
create_table(folder_detail_table_sql)
create_table(agenda_table_sql)

conn.close()

In [ ]:
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

cursor.execute(create_table_sql)
conn.commit()

for year in years:
    
    folder_year_url = f"{ujbuda.base_url}/inv/folders?year={year}"
    folder_year_response = requests.get(folder_year_url, verify=False)
    
    folder_data = folder_year_response.json()['content']
    folder_df = pd.DataFrame(folder_data)
    folder_df.columns = ['datum', 'nyilvanossagjelolo', 'kategoria', 'idopont', 'hely', 'folder_uuid']

    placeholders = ', '.join(['?'] * len(folder_df.columns))
    insert_sql = f"""
    INSERT OR IGNORE INTO {ujbuda.db_folder} 
    ({', '.join(folder_df.columns)}) 
    VALUES ({placeholders})
    """
    cursor.executemany(insert_sql, folder_df.values.tolist())
    conn.commit()

query = f"SELECT folder_uuid FROM {ujbuda.db_folder}"
folder_uuid_list = cursor.execute(query).fetchall()

conn.close()

#### Folder details, agenda points, invites


In [ ]:
folder_uuid_list = list(set(folder_uuid_list))

In [ ]:
#iterate over folder_uuid_list and get the folder details
conn = sqlite3.connect(DATABASE_PATH)
cursor = conn.cursor()

from_i = 250

for folder_uuid in folder_uuid_list[:from_i]:
    
    print(from_i)
    print(f"Folder UUID: {folder_uuid[0]}")
    
    folder_uuid = folder_uuid[0]
    
    folder_detail_url = f"{ujbuda.base_url}/detail?id={folder_uuid}"
    folder_detail_json = requests.get(folder_detail_url, verify=False)
    
    folder_detail_df = pd.DataFrame([folder_detail_json.json()["content"]])
    folder_detail_df["folder_uuid"] = folder_uuid
      
    columns = list(folder_detail_df.columns)

    for _, row in folder_detail_df.iterrows():
        uuid = row['uuid']
        cursor.execute(f"SELECT 1 FROM {ujbuda.db_detail} WHERE uuid = ?", (uuid,))
        exists = cursor.fetchone()

        if not exists:
            placeholders = ', '.join(['?'] * len(columns))
            sql = f"INSERT INTO {ujbuda.db_detail} ({', '.join(columns)}) VALUES ({placeholders})"
            cursor.execute(sql, tuple(row[col] for col in columns))

    conn.commit()
    
    # AGENDA 
    
    session_type = folder_detail_json.json()["content"]["nev"]
    print(session_type)
    session_uuid = folder_detail_json.json()["content"]["uuid"]

    print()
    if "Bizottság" in session_type:
        print("Bizottság")
        body_agenda_url = f"{ujbuda.base_url}/inv/list?id={folder_uuid}&id2={session_uuid}"
        invite_url = f"{ujbuda.base_url}/inv/biz?id={folder_uuid}"


    if session_type == "Képviselő-testület":
        print("Testület")
        body_agenda_url = f"{ujbuda.base_url}/inv/listtest?id={folder_uuid}"
        invite_url = f"{ujbuda.base_url}/inv/test?id={folder_uuid}"
        
    agenda_json = requests.get(body_agenda_url, verify=False)
    
    if not agenda_json.json()['content']:
        continue
    
    agenda_df = pd.DataFrame(agenda_json.json()["content"])
    agenda_df["folder_uuid"] = folder_uuid
    
    columns = list(agenda_df.columns) 
    
    # Get agenda points
    
    for _, row in agenda_df.iterrows():
        #print(row['targy'])
        uuid = row['uuid']
        cursor.execute(f"SELECT 1 FROM {ujbuda.db_napirendi} WHERE uuid = ?", (uuid,))
        exists = cursor.fetchone()
        if not exists:
            placeholders = ', '.join(['?'] * len(columns))
            sql = f"INSERT INTO {ujbuda.db_napirendi} ({', '.join(columns)}) VALUES ({placeholders})"
            cursor.execute(sql, tuple(row[col] for col in columns))

    conn.commit()
    
    # INVITES are downloaded on this level
    
    # Get Invite name and UUID
    try:
        invite_uuid = [point["uuid"]
            for point in agenda_json.json()["content"]
            if point["napirend"] == "0"][0]

        invite_name = [point["name"]
            for point in agenda_json.json()["content"]
            if point["napirend"] == "0"][0]
    except IndexError as ie:
        print(ie)
        continue

    invite_file_url = f"{ujbuda.base_url}/getfile/{invite_uuid}/{invite_name}"
    file_response = requests.get(invite_file_url)
    
    os.makedirs(root_folder/folder_uuid/invite_uuid, exist_ok=True)
    save_path = root_folder/folder_uuid/invite_uuid/invite_name
    
    #print(save_path)
    with open(save_path, "wb") as file:
        file.write(file_response.content)
        
    # Iterate over agenda points
    
    open_agenda_df = agenda_df[agenda_df['nyilvanossagjelolo'] == "0"]

    for _, row in open_agenda_df.iterrows():
        file_name = row["name"]
        agenda_uuid = row["uuid"]
        body_dok_url = f"{ujbuda.base_url}/elo/djav?uuid={folder_uuid}&uuid2={agenda_uuid}"

        # Get file details
        body_file_json = requests.get(body_dok_url, verify=False)
        
        if body_file_json.json()["type"] == "ERROR" or not body_file_json.json()["content"]:
            continue
            
        file_name = body_file_json.json()["content"][0]["name"]
        file_uuid = body_file_json.json()["content"][0]["uuid"]

        # Download the file
        file_download_url = f"{ujbuda.base_url}/getfile/{file_uuid}/{file_name}"
        file_response = requests.get(file_download_url)

        os.makedirs(root_folder/folder_uuid/agenda_uuid, exist_ok=True)
        save_path = root_folder/folder_uuid/agenda_uuid/file_name
        #print(save_path)

        with open(save_path, "wb") as file:
            file.write(file_response.content)
        
        #Convert to txt and keep the same structure
        doc = pymupdf.open(save_path)
        os.makedirs(output_folder/folder_uuid/agenda_uuid, exist_ok=True)
        txt_file_name = file_name.replace(".pdf",".txt")
        out = open(output_folder/folder_uuid/agenda_uuid/txt_file_name, "wb")
        for page in doc:
            text = page.get_text().encode("utf8")
            out.write(text)
            out.write(bytes((12,)))
        out.close()
        
    from_i += 1